In [79]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

In [101]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE



1.13.0+cpu



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [102]:
student_file = "./data/Processed_data.csv"
student_data = pd.read_csv(student_file)
student_data.head()
student_data.drop(student_data.columns[student_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
student_data.head()


,studentId,greV,greQ,greA,cgpa,univName,major,program,decision
0,7,157.0,147.0,4.0,3.59,New York University (NYU) - Steinhardt,Communication Sciences And Disorders,MS,Accepted
1,17,157.0,151.0,5.5,3.57,Texas A&M University,International Affairs,MS,Accepted
2,46,155.0,167.0,4.0,3.66,"University Of California, Irvine",Biotechnology Management,MS,Accepted
3,64,161.0,157.0,4.0,3.10,Boston University,Psychology,MS,Accepted
4,70,149.0,157.0,3.0,3.68,New York University (NYU) Steinhardt,Speech Language Pathology (Online),MS,Accepted


In [81]:
n_students = len(student_data)
n_uni = len((student_data['univName']).unique())
uni_names = student_data.univName.unique()
uni_data = pd.DataFrame(uni_names, columns=['univName'])
uni_data['uniId'] = range(1, n_uni + 1)
uni_data = uni_data[['uniId', 'univName']]
uni_data.head()


,uniId,univName
0,1,New York University (NYU) - Steinhardt
1,2,Texas A&M University
2,3,"University Of California, Irvine"
3,4,Boston University
4,5,New York University (NYU) Steinhardt


In [83]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

uni_dict = dict(zip(range(1, n_uni + 1), uni_names))
uni_temp = dict(zip(uni_names, range(1, n_uni + 1)))
arr = []
for i in range(n_students):
    arr.append(uni_temp.get(student_data.univName[i]))
student_dict = dict(zip(student_data.studentId, student_data.cgpa))
overall_data = pd.DataFrame(student_data.studentId, columns=['studentId'])
overall_data['GPA'] = student_data.cgpa
overall_data['uniId'] = arr
overall_data['uniName'] = student_data.univName
overall_data.head()
overall_data.to_csv('./data/overall_data.csv')


In [89]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_students, n_uni, n_factors = 20):
        super().__init__()
        # create student embeddings
        self.student_factors = torch.nn.Embedding(n_students, n_factors)
        # create university embeddings
        self.uni_factors = torch.nn.Embedding(n_uni, n_factors)
        self.student_factors.weight.data.uniform_(0, 0.05)
        self.uni_factors.weight.data.uniform_(0, 0.05)
    def forward(self, data):
        # matrix multiplication
        students, universities = data[:, 2], data[:, 3]
        return(self.student_factors(students) * self.uni_factors(universities)).sum(1)
    def predict(self, student, uni):
        return self.forward(student, uni)

In [90]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

In [91]:
class Loader(Dataset):
    def __init__(self):
        self.data = overall_data.copy()
        
        students = overall_data.studentId.unique()
        unis = overall_data.uniId.unique()
        
        print(students)
        print(unis)
        
        self.stuId2idx = {o:i for i,o in enumerate(students)}
        self.uniId2idx = {o:i for i,o in enumerate(unis)}
        
        self.idx2stuId = {i:o for o,i in self.stuId2idx.items()}
        self.idx2uniId = {i:o for o,i in self.uniId2idx.items()}
        
        self.data.uniId = overall_data.uniId.apply(lambda x: self.uniId2idx[x])
        self.data.studentId = overall_data.studentId.apply(lambda x: self.stuId2idx[x])
        
        self.data.head()
        self.x = self.data['uniId'].values
        self.y = self.data['GPA'].values
        
        self.x = torch.tensor(self.x)
        self.y = torch.tensor(self.y)
    def __getuni__(self, index):
        return(self.x[index], self.y[index])
    def __len__(self):
        return len(self.data)

In [103]:
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

model = MatrixFactorization(n_students, n_uni, n_factors=8)
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

1.13.0+cpu
[    7    17    46 ... 12644 12743 12787]
[   1    2    3 ... 3671 3672 3673]


In [104]:
# criterion = torch.nn.CrossEntropyLoss() # Define loss criterion.
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4) 
# def train():
#     model.train()
#     optimizer.zero_grad() # Clear gradients.
#     out = model(data.x) # Perform a single forward pass.
#     loss = criterion(out[data.train_mask], data.y[data.train_mask])
#     loss.backward() # Derive gradients.
#     optimizer.step() # Update parameters based on gradients.
#     return loss

# for epoch in range(1, 201):
#     loss = train()
# print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
cuda = torch.cuda.is_available()
print("Is running on GPU:", cuda)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

Is running on GPU: False
student_factors.weight tensor([[0.0376, 0.0302, 0.0285,  ..., 0.0113, 0.0315, 0.0244],
        [0.0131, 0.0468, 0.0085,  ..., 0.0453, 0.0039, 0.0450],
        [0.0098, 0.0195, 0.0034,  ..., 0.0181, 0.0169, 0.0152],
        ...,
        [0.0480, 0.0060, 0.0181,  ..., 0.0114, 0.0240, 0.0150],
        [0.0065, 0.0232, 0.0327,  ..., 0.0457, 0.0234, 0.0459],
        [0.0093, 0.0086, 0.0336,  ..., 0.0017, 0.0244, 0.0068]])
uni_factors.weight tensor([[0.0421, 0.0305, 0.0059,  ..., 0.0423, 0.0295, 0.0431],
        [0.0126, 0.0090, 0.0149,  ..., 0.0379, 0.0230, 0.0458],
        [0.0401, 0.0418, 0.0461,  ..., 0.0280, 0.0255, 0.0090],
        ...,
        [0.0145, 0.0066, 0.0380,  ..., 0.0320, 0.0259, 0.0499],
        [0.0261, 0.0479, 0.0339,  ..., 0.0240, 0.0129, 0.0182],
        [0.0174, 0.0376, 0.0415,  ..., 0.0184, 0.0313, 0.0010]])


AssertionError: Torch not compiled with CUDA enabled

In [95]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
trained_uni_embeddings = model.uni_factors.weight.data.cpu().numpy()
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_uni_embeddings)
for cluster in range(5):
    print("Cluster #{}".format(cluster))
    unis = []
    for uniIdx in np.where(kmeans.labels_ == cluster)[0]:
        uniId = train_set.idx2uniId[uniIdx]
        gpa_count = overall_data.loc[overall_data['uniId']==uniId].count()[0]
        unis.append((uni_dict[uniId], gpa_count))
    for uni in sorted(unis, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", uni[0])

Cluster #0
	 University Of Michigan
	 Northwestern University
	 Purdue University
	 Georgetown University
	 University Of Houston
	 University Of Maryland
	 University Of Alberta
	 Yale University
	 University Of Kentucky
	 University Of California, Berkeley (UCB)
Cluster #1
	 Columbia University
	 Cornell University
	 UCLA
	 University Of Pittsburgh
	 University Of South Florida
	 Washington University in St. Louis (WashU/WUSTL)
	 New York University (NYU)
	 University Of Arizona
	 Western Michigan University
	 Columbia  University
Cluster #2
	 University Of Pennsylvania
	 New York University
	 University Of Texas At Dallas
	 University Of Virginia
	 University Of Central Florida
	 Johns Hopkins University
	 University Of Georgia
	 Clemson University
	 University Of Kansas
	 Illinois State University
Cluster #3
	 Arizona State University
	 Carnegie Mellon University (CMU)
	 Towson University
	 University Of Oxford
	 ีUniversity Of Maryland - College Park (UMD)
	 University Of Texas At